# QLoRA 파인튜닝 워크플로우 (10단계)
- 작성일: 2025-10-23 00:46:20
- 대상: Colab/Kaggle T4(1장) 또는 AWS g4dn.xlarge(T4×1) / 멀티-GPU는 DDP
- 프레임워크: `unsloth + transformers + peft + accelerate + bitsandbytes`

이 노트북은 **대회 환경에서 빠르게 QLoRA 파인튜닝**을 진행하기 위한 최소·필수 단계를 10셀로 구성했습니다.
각 셀은 독립 실행 가능하며, 순서대로 실행하면 학습 → 검증 → 추론까지 완료됩니다.


## 1) 환경/설정 → 시드·경로·하드웨어(T4×2, 4bit, bf16)

In [ ]:
%%bash
# (선택) 런타임에 설치가 필요한 경우 주석 해제 후 실행하세요.
# pip install -q "unsloth[cu121]" transformers peft bitsandbytes accelerate datasets trl rouge-score evaluate
# pip install -q scikit-learn pandas matplotlib


In [ ]:
# 기본 설정
import os, random, numpy as np, torch
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

PROJ = './qlora_run'
os.makedirs(PROJ, exist_ok=True)

n_gpu = torch.cuda.device_count()
print('GPU count:', n_gpu)
if n_gpu:
    for i in range(n_gpu):
        print(f'CUDA:{i}', torch.cuda.get_device_name(i))
BF16 = torch.cuda.is_bf16_supported()
print('BF16 supported:', BF16)


## 2) 데이터 인덱싱 → 파일 리스트업, 중복/깨진 샘플 스킵
- 입력 포맷 예시: `train.jsonl` (필수: `instruction`, `output`; 선택: `source`, `lang`)

In [ ]:
import json, hashlib

TRAIN_JSONL = 'train.jsonl'  # 경로를 상황에 맞게 수정

def stream_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                yield json.loads(line)

def normalize_text(s: str) -> str:
    return ' '.join(s.split())

seen = set(); data = []
for ex in stream_jsonl(TRAIN_JSONL):
    if not ex.get('instruction') or not ex.get('output'):
        continue
    key = hashlib.md5((normalize_text(ex['instruction']) + normalize_text(ex['output'])).encode()).hexdigest()
    if key in seen:
        continue
    seen.add(key); data.append(ex)
print('Loaded samples:', len(data))


## 3) EDA → 길이 분포/버킷(층화용), 샘플 확인

In [ ]:
from transformers import AutoTokenizer
import numpy as np

BASE_MODEL = 'unsloth/phi-3-mini-4k-instruct-bnb-4bit'  # 또는 llama-3-8b-instruct 4bit 가용 환경
tok = AutoTokenizer.from_pretrained(BASE_MODEL)

PROMPT_SYS = 'You are a helpful assistant.'
def count_tokens(ex):
    s = f"<s>[SYSTEM]{PROMPT_SYS}\n[USER]{ex['instruction']}\n[ASSISTANT]{ex['output']}</s>"
    return len(tok(s, add_special_tokens=False)['input_ids'])

lens = [count_tokens(x) for x in data]
pcts = np.percentile(lens, [50,90,95,99]).tolist()
print('Token length p50/90/95/99:', pcts, 'max:', max(lens) if lens else None)

def bucketize(n):
    # [<=512:0, <=1024:1, <=1536:2, <=2048:3, >2048:4]
    if n <= 512: return 0
    if n <= 1024: return 1
    if n <= 1536: return 2
    if n <= 2048: return 3
    return 4

buckets = [bucketize(l) for l in lens]
from collections import Counter
print('Length bucket dist:', Counter(buckets))

# (선택) 히스토그램 시각화
import matplotlib.pyplot as plt
plt.figure()
plt.hist(lens, bins=30)
plt.title('Token length histogram')
plt.xlabel('tokens'); plt.ylabel('count')
plt.show()


## 4) 전처리/포맷 → 템플릿 통일, 라벨 마스킹(-100), 시퀀스 패킹

In [ ]:
MAX_LEN = 1024  # 여유 시 1536/2048

def build_ids_and_labels(ex):
    user = f"<s>[SYSTEM]{PROMPT_SYS}\n[USER]{ex['instruction']}\n[ASSISTANT]"
    ans  = ex['output'] + tok.eos_token
    u = tok(user, add_special_tokens=False)
    a = tok(ans, add_special_tokens=False)
    input_ids = u['input_ids'] + a['input_ids']
    labels    = [-100]*len(u['input_ids']) + a['input_ids']
    attn_mask = [1]*len(input_ids)
    return {'input_ids': input_ids, 'labels': labels, 'attention_mask': attn_mask}

proc = [build_ids_and_labels(x) for x in data]

def pack(samples, max_len=MAX_LEN):
    out=[]
    cur={'input_ids':[], 'labels':[], 'attention_mask':[]}
    for s in samples:
        L = len(s['input_ids'])
        if L > max_len:  # 너무 긴 샘플은 잘라내기
            s={'input_ids': s['input_ids'][:max_len],
               'labels': s['labels'][:max_len],
               'attention_mask': s['attention_mask'][:max_len]}
        if len(cur['input_ids']) + len(s['input_ids']) > max_len and cur['input_ids']:
            out.append(cur)
            cur={'input_ids':[], 'labels':[], 'attention_mask':[]}
        for k in cur:
            cur[k] += s[k]
    if cur['input_ids']:
        out.append(cur)
    return out

print('Example packed length check: before', len(proc), '→ after (by bucket later)')


## 5) 커스텀 Dataset/Dataloader 구성

In [ ]:
import torch
class ListDataset(torch.utils.data.Dataset):
    def __init__(self, xs): self.xs = xs
    def __len__(self): return len(self.xs)
    def __getitem__(self, i):
        return {k: torch.tensor(v) for k,v in self.xs[i].items()}


## 6) 모델 빌더 → Unsloth 4bit 로드 + LoRA 타깃 모듈

In [ ]:
from unsloth import FastLanguageModel
from peft import LoraConfig, get_peft_model

base, tok = FastLanguageModel.from_pretrained(
    BASE_MODEL,
    load_in_4bit=True,
    use_gradient_checkpointing=True,
)

peft_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.1, bias='none',
    target_modules=['q_proj','v_proj','k_proj','o_proj','gate_proj','up_proj','down_proj'],
    task_type='CAUSAL_LM',
)
model = get_peft_model(base, peft_cfg)
print('Model ready (LoRA attached).')


## 7) 학습 유틸 → 손실/지표, bf16, EarlyStopping, Cosine 스케줄러

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers.trainer_callback import EarlyStoppingCallback

collator = DataCollatorForLanguageModeling(tok, mlm=False)

args = TrainingArguments(
    output_dir=f"{PROJ}/ckpt",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    num_train_epochs=2,
    lr_scheduler_type='cosine',
    warmup_ratio=0.03,
    weight_decay=0.0,
    logging_steps=25,
    evaluation_strategy='steps', eval_steps=200,
    save_strategy='steps', save_steps=200, save_total_limit=3,
    bf16=BF16, fp16=not BF16,
    gradient_checkpointing=True, max_grad_norm=1.0,
    dataloader_num_workers=2,
    report_to='none',
)

print('TrainingArguments ready.')


## 8) Stratified K-Fold 학습(길이 버킷 기준 3~5-Fold)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np, random

X = np.arange(len(proc)); y = np.array(buckets)
SK = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

fold_paths = []
for fold, (tr_idx, va_idx) in enumerate(SK.split(X, y), 1):
    tr = [proc[i] for i in tr_idx]; va = [proc[i] for i in va_idx]
    # 길이 버킷별로 섞은 뒤 패킹(간단 구현)
    def by_bucket(items, idxs):
        return [items[i] for i in idxs]
    tr_pack = pack(tr, MAX_LEN); va_pack = pack(va, MAX_LEN)
    train_ds = ListDataset(tr_pack); val_ds = ListDataset(va_pack)

    fold_out = f"{PROJ}/fold{fold}"
    os.makedirs(fold_out, exist_ok=True)

    fold_args = args.clone()
    fold_args.output_dir = fold_out

    trainer = Trainer(
        model=model,
        args=fold_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=collator,
        tokenizer=tok,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    # 주의: 실제 학습은 시간이 오래 걸립니다. 필요시 주석 해제.
    # trainer.train()
    # trainer.save_model(f"{fold_out}/adapter")
    
    fold_paths.append(fold_out)

print('Fold output paths:', fold_paths)


## 9) 테스트셋 평가(최대 2,000문항) → 리포트/샘플 예측

In [ ]:
from tqdm.auto import tqdm
import pandas as pd

TEST_JSONL = 'test.jsonl'  # 경로 수정

def stream_test(path):
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line=line.strip()
            if line:
                yield json.loads(line)

def generate_answer(prompt, max_new_tokens=256):
    s = f"<s>[SYSTEM]{PROMPT_SYS}\n[USER]{prompt}\n[ASSISTANT]"
    ids = tok(s, return_tensors='pt')
    device = next(model.parameters()).device
    ids = {k:v.to(device) for k,v in ids.items()}
    model.eval()
    with torch.inference_mode():
        out = model.generate(**ids, max_new_tokens=max_new_tokens, do_sample=False)
    text = tok.decode(out[0], skip_special_tokens=True)
    return text.split('[ASSISTANT]')[-1].strip()

pred_rows=[]
# 실제 생성은 시간이 오래 걸릴 수 있습니다. 필요시 제한하세요.
# for i, ex in enumerate(stream_test(TEST_JSONL)):
#     if i>=2000: break
#     pred = generate_answer(ex['instruction'])
#     pred_rows.append({'instruction': ex['instruction'], 'prediction': pred})

df = pd.DataFrame(pred_rows)
csv_path = f"{PROJ}/preds.csv"
if len(df):
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print('Saved:', csv_path)
else:
    print('생성 예시가 비어 있습니다. 상단 루프 주석을 해제 후 실행하세요.')


## 10) 결과 취합/베스트 저장 + (옵션) LoRA 병합(merge)

In [ ]:
from unsloth import FastLanguageModel

# 어댑터만 사용할 경우: fold별 adapter 디렉토리 보관
# (옵션) 병합하여 단일 추론 가중치로 저장
# 주의: 병합은 FP16 메모리 필요

# merged = FastLanguageModel.merge_and_unload(model)
# merged.save_pretrained(f"{PROJ}/merged_full_fp16")
# tok.save_pretrained(f"{PROJ}/merged_full_fp16")
print('Done. 필요 시 병합 코드를 주석 해제하세요.')


---
### ✅ 오버피팅/언더피팅 빠른 처방
- **오버피팅**: `lora_dropout 0.1→0.3`, `r 16→8`, 학습 스텝↓, 템플릿 통일, 중복 제거
- **언더피팅**: `r 16→32`, target_modules 확대, steps↑, 데이터 클린업/보강
- **OOM**: `MAX_LEN↓(1024→768)`, grad_accum↑, rank↓, 패킹 유지

### 🔧 멀티 GPU(T4×2)
```bash
accelerate config   # distributed_type=NO(DDP) 또는 AUTO, mixed_precision=bf16
accelerate launch your_train_script.py
```

### 📦 체크포인트 관리
- `save_total_limit=3`로 용량 절약
- **LoRA 어댑터만 저장** 권장 (배포/제출 시 용량↓)
